In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import scipy.stats as stats

In [82]:
df=pd.read_csv('./Substantial_environment_carbonpred_variables_20240907.csv',index_col='Date')
df.index=pd.to_datetime(df.index)

In [83]:
df_revised=df.copy()

先用當前缺失值的前五筆資料做rolling 若還是有缺失值則用向前填充法

In [84]:
for col in df_revised.columns[df_revised.isna().sum() > 0]:   
    df_revised[col] = df_revised[col].where(df_revised[col].notna(), 
    df_revised[col].rolling(window=5, min_periods=1).mean().round(3).fillna(method='ffill'))


C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1937381058.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_revised[col].rolling(window=5, min_periods=1).mean().round(3).fillna(method='ffill'))
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1937381058.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_revised[col].rolling(window=5, min_periods=1).mean().round(3).fillna(method='ffill'))
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1937381058.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_revised[col].rolling(window=5, min_periods=1).mean().round(3).fillna(method='ffill'))
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1937381058.py:3: FutureWarning: Series.fillna with 'method' is deprecated and

有些future的開頭前100筆即為缺失值，為避免吃到未來資料(test_data)，使用5筆資料的平均填補最前面100筆的缺失值

In [85]:
for col in df_revised.columns[df_revised.isna().any()]:  
    df_revised[col] = df_revised[col].fillna(df[col].loc['2022-11-18':"2022-11-24"].mean().round(3))

In [ ]:
# 使用 Kolmogorov-Smirnov 檢驗填補前與填補後是否有顯著差異
for col in df.columns:
    
    ks_stat, p_value = stats.ks_2samp(df[col][~np.isnan(df[col])], df_revised[col])
    
    if p_value > 0.05:
        continue
    else:
        print(f'{col}:')
        print(f'{df[col].isna().sum()}')
        print("p 值:", p_value)
        print("填补后的数据分布与原始数据分布有显著差异（拒绝原假设）")


ygj24.Price:
103
p 值: 1.252224079779694e-07
填补后的数据分布与原始数据分布有显著差异（拒绝原假设）
zgh24.Price:
103
p 值: 2.972925237572645e-08
填补后的数据分布与原始数据分布有显著差异（拒绝原假设）
yih24.Price:
103
p 值: 3.3278647031710624e-08
填补后的数据分布与原始数据分布有显著差异（拒绝原假设）
zih24.Price:
103
p 值: 3.3278647031710624e-08
填补后的数据分布与原始数据分布有显著差异（拒绝原假设）


將每筆資料做lag，分別共 lag 12天

In [87]:
for col in df_revised.columns:
    for i in range(1,13):
        df_revised[f'{col}_lag{i}']=df_revised[col].shift(i)

C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\4091287530.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_revised[f'{col}_lag{i}']=df_revised[col].shift(i)
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\4091287530.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_revised[f'{col}_lag{i}']=df_revised[col].shift(i)
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\4091287530.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

如果偏態大於1，表示有中度偏態，取log()能壓縮較大的值

In [88]:
for col in df_revised.columns:
    col_skew=df_revised[col].skew()
    if col_skew >=1:
        df_revised[f'log({col})']=np.log(df_revised[col]+1).round(3)

C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\2627546674.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_revised[f'log({col})']=np.log(df_revised[col]+1).round(3)
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\2627546674.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_revised[f'log({col})']=np.log(df_revised[col]+1).round(3)
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\2627546674.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

In [89]:
def MA(target, short, long):
    
    short_ma = target.rolling(window=short).mean()
    long_ma = target.rolling(window=long).mean()

    return short_ma, long_ma

In [90]:
def calculate_moving_average_indicators(prices, short_windows, long_windows):
    
    ma_df = pd.DataFrame(index=prices.index)

    for short in short_windows:
        for long in long_windows:
            short_ma, long_ma = MA(prices, short, long)
            ma_df[f'MA({short},{long})'] = (short_ma - long_ma).round(3)

    return ma_df


計算MA()

In [91]:
# Define short and long windows
short_windows = [1, 2, 3]
long_windows = [9, 12]

# Calculate moving average indicators
moving_average_indicators = calculate_moving_average_indicators(df_revised['carbon.Price'], short_windows, long_windows)

# Combine original prices and moving average indicators
df_revised = pd.concat([df_revised, moving_average_indicators], axis=1)

In [92]:
def calculate_momentum_indicators(prices, periods):
   
    momentum_df = pd.DataFrame(index=prices.index)

    for period in periods:
        momentum_df[f'MOM({period})'] = (prices - prices.shift(period)).round(3)

    return momentum_df


計算MOM

In [93]:
# Define momentum periods
momentum_periods = [1, 2, 3, 6, 9, 12]

# Calculate momentum indicators
momentum_indicators = calculate_momentum_indicators(df_revised['carbon.Price'], momentum_periods)

# Combine original prices and MOM indicators
df_revised = pd.concat([df_revised,momentum_indicators], axis=1)

In [94]:
def calculate_obv(prices, volumes):
    
    if len(prices) != len(volumes):
        raise ValueError("Prices and volumes must have the same length")
    
    if len(prices) == 0:
        return pd.Series()
        
    obv = [0]
    for i in range(1, len(prices)):
        if prices[i] > prices[i - 1]:
            obv.append(obv[-1] + volumes[i])
        elif prices[i] < prices[i - 1]:
            obv.append(obv[-1] - volumes[i])
        else:
            obv.append(obv[-1])
            
    return pd.Series(obv, index=prices.index)

In [96]:
def calculate_obv_moving_averages(df, short_windows, long_windows):
    
    if 'carbon.Price' not in df or 'carbon.Vol' not in df:
        raise ValueError("DataFrame must contain 'Price' and 'Volume' columns")
        
    obv = calculate_obv(df['carbon.Price'], df['carbon.Vol'])
    
    for short in short_windows:
        for long in long_windows:
            if short >= long:
                continue
            
            # 計算移動平均值
            short_ma, long_ma = MA(obv, short, long)
            
            # 將結果加入到 df
            df[f"OBV_({short}, {long})"] = (short_ma - long_ma).round(3)
    
    return df  

計算OBV

In [97]:
short_windows = [1, 2, 3]
long_windows = [9, 12]
# Calculate OBV with moving average combinations
df_revised = calculate_obv_moving_averages(df_revised, short_windows, long_windows)

C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1537045027.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if prices[i] > prices[i - 1]:
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1537045027.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif prices[i] < prices[i - 1]:
C:\Users\tammy\AppData\Local\Temp\ipykernel_26792\1537045027.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obv.append(obv[-1] - volumes[i])

因為lag的關係，把前12筆NA丟掉

In [98]:
df_revised=df_revised.dropna()

In [99]:
df_revised.to_csv('carbonpred_variables.csv')